<a href="https://colab.research.google.com/github/ShubhamK155/Retail-Orders-Analysis/blob/main/Retail_Orders_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracting data from kaggle API**




In [ ]:
# extracting data
!kaggle datasets download -d ankitbansal06/retail-orders

In [ ]:
#Extracting file from zip file
!unzip retail-orders.zip

In [ ]:
# importing libraries
import pandas as pd

# Loading CSV file into DataFrame
df = pd.read_csv('/content/orders.csv')

In [ ]:
# reading dataframe
df

**Data Transformation, Cleaning & Manipulation**

In [ ]:
# null handling
import numpy as np

In [ ]:
# checking for unique values for each column

col_name = df.columns

for index in col_name:
  print(f"'{index}': {df[index].unique()}")

# Got 'Ship Mode': ['Second Class' 'Standard Class' 'Not Available' 'unknown' 'First Class']
# Changing 'Not Available' 'unknown' to Null Values

In [ ]:
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])

In [ ]:
col_name = df.columns

for index in col_name:
  print(f"'{index}': {df[index].unique()}")

# verifying the change 'Not Available' 'unknown' to Null Values

In [ ]:
# checking every column for null values

for index in col_name:
  if df[index].isnull().any():
    print({index})

# Above will return {'Ship Mode'} has null values

In [ ]:
col_name
#returns a list of column names

In [ ]:
#renaming the column names make them lower case and replace space with underscore

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns

In [ ]:
df

In [ ]:
#deriving new columns discount, sale price and profit

df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']= df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']

In [ ]:
#converting order date from object data type to datetime

df['order_date'] = pd.to_datetime(df['order_date'], format = "%Y-%m-%d")

In [ ]:
df.info()

In [ ]:
#dropping cost price list price and discount percent columns
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

In [ ]:
df

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,3,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,2,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,4,0.9,29.1,-0.9


**Loading Data in SQLite**

In [ ]:
# Loading the Data in Sqlite
import sqlite3

# Connecting to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('database.db')

# Creating a table in SQLite (schema inferred from DataFrame)
df.to_sql('orders_final', conn, if_exists = 'replace', index = False)

c = conn.cursor()

In [ ]:
# Verifying the ETL pipeline

c.execute("SELECT discount,sale_price,profit FROM orders_final limit 10")
rows = c.fetchall()
for row in rows:
  print(row)

(5.2, 254.8, 14.800000000000011)
(21.900000000000002, 708.1, 108.10000000000002)
(0.5, 9.5, -0.5)
(19.2, 940.8, 160.79999999999995)
(1.0, 19.0, -1.0)
(1.5, 48.5, -1.5)
(0.3, 9.7, -0.3000000000000007)
(45.5, 864.5, 4.5)
(0.4, 19.6, -0.3999999999999986)
(3.3000000000000003, 106.7, 16.700000000000003)


**Exploratory data analysis using SQL**

In [ ]:
# top 10 highest reveue generating products
c.execute("""
    SELECT product_id, SUM(sale_price) AS sales
    FROM orders_final
    GROUP BY product_id
    ORDER BY sales DESC
    LIMIT 10
""")
rows = c.fetchall()
for row in rows:
  print(row)

('TEC-CO-10004722', 59514.0)
('OFF-BI-10003527', 26525.300000000003)
('TEC-MA-10002412', 21734.4)
('FUR-CH-10002024', 21096.2)
('OFF-BI-10001359', 19090.2)
('OFF-BI-10000545', 18249.0)
('TEC-CO-10001449', 18151.2)
('TEC-MA-10001127', 17906.4)
('OFF-BI-10004995', 17354.8)
('OFF-SU-10000151', 16325.8)


In [ ]:
#top 5 highest selling products in each region
c.execute("""
WITH cte AS (
    SELECT
        region,
        product_id,
        SUM(sale_price) AS sales
    FROM
        orders_final
    GROUP BY
        region, product_id
),
ranked_cte AS (
    SELECT
        region,
        product_id,
        sales,
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY sales DESC) AS rn
    FROM
        cte
)
SELECT
    region,
    product_id,
    sales
FROM
    ranked_cte
WHERE
    rn <= 5
""")

rows = c.fetchall()
for row in rows:
  print(row)


('Central', 'TEC-CO-10004722', 16975.0)
('Central', 'TEC-MA-10000822', 13770.0)
('Central', 'OFF-BI-10001120', 11056.5)
('Central', 'OFF-BI-10000545', 10132.7)
('Central', 'OFF-BI-10004995', 8416.1)
('East', 'TEC-CO-10004722', 29099.0)
('East', 'TEC-MA-10001047', 13767.0)
('East', 'FUR-BO-10004834', 11274.1)
('East', 'OFF-BI-10001359', 8463.599999999999)
('East', 'TEC-CO-10001449', 8316.0)
('South', 'TEC-MA-10002412', 21734.4)
('South', 'TEC-MA-10001127', 11116.4)
('South', 'OFF-BI-10001359', 8053.200000000001)
('South', 'TEC-MA-10004125', 7840.0)
('South', 'OFF-BI-10003527', 7391.4)
('West', 'TEC-CO-10004722', 13440.0)
('West', 'OFF-SU-10000151', 12592.3)
('West', 'FUR-CH-10001215', 9604.0)
('West', 'OFF-BI-10003527', 7804.799999999999)
('West', 'TEC-AC-10003832', 7722.7)


In [ ]:
# month over month growth comparison for 2022 and 2023 sales eg : jan 2022 vs jan 2023
c.execute("""
    WITH cte AS (
        SELECT
            strftime('%Y', order_date) AS order_year,
            strftime('%m', order_date) AS order_month,
            SUM(sale_price) AS sales
        FROM orders_final
        GROUP BY order_year, order_month
    )
    SELECT
        order_month,
        SUM(CASE WHEN order_year = '2022' THEN sales ELSE 0 END) AS sales_2022,
        SUM(CASE WHEN order_year = '2023' THEN sales ELSE 0 END) AS sales_2023
    FROM cte
    GROUP BY order_month
    ORDER BY order_month
""")
rows = c.fetchall()
for row in rows:
  print(row)


('01', 94712.49999999997, 88632.6)
('02', 90091.0, 128124.20000000011)
('03', 80105.99999999996, 82512.29999999994)
('04', 95451.60000000005, 111568.60000000006)
('05', 79448.29999999993, 86447.89999999994)
('06', 94170.49999999999, 68976.5)
('07', 78652.20000000003, 90563.79999999993)
('08', 104807.99999999996, 87733.59999999999)
('09', 79142.19999999991, 76658.59999999993)
('10', 118912.69999999998, 121061.49999999993)
('11', 84225.29999999997, 75432.79999999993)
('12', 95869.90000000004, 102556.09999999999)


In [ ]:
#for each category which month had highest sales
c.execute("""
    WITH cte AS (
        SELECT
            category,
            strftime('%Y-%m', order_date) AS order_year_month,
            SUM(sale_price) AS sales
        FROM orders_final
        GROUP BY category, order_year_month
    )
    SELECT * FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales DESC) AS rn
        FROM cte
    ) a
    WHERE rn = 1
""")
rows = c.fetchall()
for row in rows:
  print(row)

('Furniture', '2022-10', 42888.90000000001, 1)
('Office Supplies', '2023-02', 44118.499999999985, 1)
('Technology', '2023-10', 53000.10000000002, 1)


In [ ]:
#which sub category had highest growth by profit in 2023 compare to 2022
c.execute("""
    WITH cte AS (
        SELECT
            sub_category,
            strftime('%Y', order_date) AS order_year,
            SUM(sale_price) AS sales
        FROM orders_final
        GROUP BY sub_category, order_year
    ),
    cte2 AS (
        SELECT
            sub_category,
            SUM(CASE WHEN order_year = '2022' THEN sales ELSE 0 END) AS sales_2022,
            SUM(CASE WHEN order_year = '2023' THEN sales ELSE 0 END) AS sales_2023
        FROM cte
        GROUP BY sub_category
    )
    SELECT
        sub_category,
        sales_2022,
        sales_2023,
        (sales_2023 - sales_2022) AS sales_difference
    FROM cte2
    ORDER BY sales_difference DESC
    LIMIT 1
""")
rows = c.fetchall()
for row in rows:
  print(row)

('Machines', 73723.2, 109178.50000000001, 35455.30000000002)
